In [1]:
import cv2
import numpy as np

In [2]:
PATH = 'pedestrian'
N = 60
BUFOR_SIZE = 30

In [3]:
f = open(f'{PATH}/temporalROI.txt','r') 
line = f.readline()
roi_start , roi_end = line.split() 
roi_start = int(roi_start) 
roi_end = int(roi_end)

I_start = cv2.imread(f'{PATH}/input/in%06d.jpg' % roi_start)
HEIGHT, WIDTH, _ = I_start.shape

In [4]:
def gray_and_type(I):  # noqa: E741
    I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)  # noqa: E741
    I = I.astype('uint8')  # noqa: E741
    return I

In [5]:
def binarization(I, threshold=15):  # noqa: E741
    (_, thresh) = cv2.threshold(I, threshold,255,cv2.THRESH_BINARY)
    return thresh

In [6]:
def morphological_operations(B, kernel_size=3, blur_size=3, erode_iter=1, dilate_iter=2):
    kernel = np.ones((kernel_size, kernel_size),np.uint8)
    M = cv2.medianBlur(B, blur_size) 
    M = cv2.erode(M, kernel, iterations=erode_iter)
    M = cv2.dilate(M, kernel, iterations=dilate_iter)
    return M

In [7]:
def calculate_metrics(M, ref_mask, TP, TN, FP, FN):
    TP_M = np.logical_and((M==255), (ref_mask==255))
    TP = TP + np.sum(TP_M)

    TN_M = np.logical_and((M==0), (ref_mask==255))
    TN = TN + np.sum(TN_M)

    FP_M = np.logical_and((M==255), (ref_mask==0))
    FP = FP + np.sum(FP_M)

    FN_M = np.logical_and((M==0), (ref_mask==255))
    FN = FN + np.sum(FN_M)
    return TP, TN, FP, FN

### Metody oparte o bufor próbek

In [10]:
for operation in [np.mean, np.median]:
    TP, TN, FP, FN, iN = 0, 0, 0, 0, 0
    BUF = np.zeros((HEIGHT, WIDTH, N), dtype=np.uint8)

    for i in range(roi_start, roi_end, 1):
        img = cv2.imread(f'{PATH}/input/in%06d.jpg' % i)
        IG = gray_and_type(img)
        BUF[:,:,iN] = IG
        iN = (iN + 1) % N
        I_operation = operation(BUF, axis=2).astype('uint8')

        I_diff = cv2.absdiff(IG, I_operation).astype('uint8')

        B = binarization(I_diff, threshold=22)
        M = morphological_operations(B)

        ref_mask = cv2.imread(f'{PATH}/groundtruth/gt%06d.png' % i, cv2.IMREAD_GRAYSCALE)


        TP, TN, FP, FN = calculate_metrics(M, ref_mask, TP, TN, FP, FN)
    
        # cv2.imshow('operation', M)
        # cv2.waitKey(3)
    P = TP / (TP + FP)
    R = TP / (TP + FN)
    F1 = 2 * P * R / (P + R)
    print(f'{operation.__name__} precision: ', P, 'Recall: ', R, 'F1: ', F1)

(240, 360)
[[2 2 2 ... 3 3 3]
 [2 2 2 ... 3 3 3]
 [2 2 2 ... 4 4 4]
 ...
 [2 2 2 ... 3 3 3]
 [2 2 2 ... 3 3 3]
 [2 2 2 ... 3 3 3]]


### Aproksymacja średniej i mediany (tzw. sigma-delta)

In [9]:
TP, TN, FP, FN = 0, 0, 0, 0

BG = cv2.imread(f'{PATH}/input/in%06d.jpg' % roi_start, cv2.IMREAD_GRAYSCALE).astype('float64')
alfa = 0.01

for i in range(roi_start + 1, roi_end, 1):
    img = cv2.imread(f'{PATH}/input/in%06d.jpg' % i)
    IG = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype('float64')

    BG = IG * alfa + ( 1 - alfa ) * BG

    I_diff = cv2.absdiff(IG, BG).astype('uint8')

    B = binarization(I_diff, threshold=20)
    M = morphological_operations(B)

    ref_mask = cv2.imread(f'{PATH}/groundtruth/gt%06d.png' % i, cv2.IMREAD_GRAYSCALE)


    TP, TN, FP, FN = calculate_metrics(M, ref_mask, TP, TN, FP, FN)

    # cv2.imshow('operation', M)
    # cv2.waitKey(3)

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)
print(f'{np.mean.__name__} precision: ', P, 'Recall: ', R, 'F1: ', F1)

mean precision:  0.8893925533698789 Recall:  0.972451860442237 F1:  0.9290695149087672


In [10]:
TP, TN, FP, FN = 0, 0, 0, 0
BG = cv2.imread(f'{PATH}/input/in%06d.jpg' % roi_start, cv2.IMREAD_GRAYSCALE)
alfa = 0.01

for i in range(roi_start + 1, roi_end, 1):
    img = cv2.imread(f'{PATH}/input/in%06d.jpg' % i)
    IG = gray_and_type(img)

    BG = np.where(IG > BG, BG + 1, BG)
    BG = np.where(IG < BG, BG - 1, BG)


    I_diff = cv2.absdiff(IG, BG).astype('uint8')

    B = binarization(I_diff, threshold=20)
    M = morphological_operations(B)

    ref_mask = cv2.imread(f'{PATH}/groundtruth/gt%06d.png' % i, cv2.IMREAD_GRAYSCALE)


    TP, TN, FP, FN = calculate_metrics(M, ref_mask, TP, TN, FP, FN)

    # cv2.imshow('operation', I_diff)
    # cv2.waitKey(3)

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)
print(f'{np.median.__name__} precision: ', P, 'Recall: ', R, 'F1: ', F1)

median precision:  0.9436593811960415 Recall:  0.9712280618070482 F1:  0.9572452680223577


### Polityka aktualizacji

In [11]:
TP, TN, FP, FN = 0, 0, 0, 0
BG = cv2.imread(f'{PATH}/input/in%06d.jpg' % roi_start, cv2.IMREAD_GRAYSCALE)
alfa = 0.01

for i in range(roi_start + 1, roi_end, 1):
    img = cv2.imread(f'{PATH}/input/in%06d.jpg' % i)
    IG = gray_and_type(img)

    I_diff = cv2.absdiff(IG, BG).astype('uint8')

    B = binarization(I_diff, threshold=20)
    M = morphological_operations(B)

    ref_mask = cv2.imread(f'{PATH}/groundtruth/gt%06d.png' % i, cv2.IMREAD_GRAYSCALE)

    BG = np.where(np.logical_and(IG > BG, M==0), BG + 1, BG)
    BG = np.where(np.logical_and(IG < BG, M==0), BG - 1, BG)


    TP, TN, FP, FN = calculate_metrics(M, ref_mask, TP, TN, FP, FN)

    # cv2.imshow('operation', M)
    # cv2.waitKey(3)

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)
print(f'{np.median.__name__} precision: ', P, 'Recall: ', R, 'F1: ', F1)

median precision:  0.9328398720515213 Recall:  0.9759220227110792 F1:  0.953894749571467


In [12]:
TP, TN, FP, FN = 0, 0, 0, 0

BG = cv2.imread(f'{PATH}/input/in%06d.jpg' % roi_start, cv2.IMREAD_GRAYSCALE).astype('float64')
alfa = 0.01

for i in range(roi_start + 1, roi_end, 1):
    img = cv2.imread(f'{PATH}/input/in%06d.jpg' % i)
    IG = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype('float64')

    I_diff = cv2.absdiff(IG, BG).astype('uint8')

    B = binarization(I_diff, threshold=20)
    M = morphological_operations(B)

    ref_mask = cv2.imread(f'{PATH}/groundtruth/gt%06d.png' % i, cv2.IMREAD_GRAYSCALE)

    BG = np.where(M==0, IG * alfa + ( 1 - alfa ) * BG, BG)

    TP, TN, FP, FN = calculate_metrics(M, ref_mask, TP, TN, FP, FN)

    # cv2.imshow('operation', M)
    # cv2.waitKey(3)

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)
print(f'{np.mean.__name__} precision: ', P, 'Recall: ', R, 'F1: ', F1)

mean precision:  0.9189015401462929 Recall:  0.9756194269462274 F1:  0.946411477718685


### OpenCV – GMM/MOG

In [13]:
TP, TN, FP, FN = 0, 0, 0, 0

fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=30, detectShadows=False)

for i in range(roi_start + 1, roi_end, 1):
    img = cv2.imread(f'{PATH}/input/in%06d.jpg' % i)
    IG = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    fgmask = fgbg.apply(IG, learningRate=0.01)


    ref_mask = cv2.imread(f'{PATH}/groundtruth/gt%06d.png' % i, cv2.IMREAD_GRAYSCALE)


    TP, TN, FP, FN = calculate_metrics(fgmask, ref_mask, TP, TN, FP, FN)

    # cv2.imshow('operation', fgmask)
    # cv2.waitKey(3)

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)
print('MOG precision: ', P, 'Recall: ', R, 'F1: ', F1)

MOG precision:  0.7622707930367505 Recall:  0.8811663203460622 F1:  0.8174177695487023


### OpenCV – KNN

In [14]:
TP, TN, FP, FN = 0, 0, 0, 0

fgbg = cv2.createBackgroundSubtractorKNN(history=500, dist2Threshold=30, detectShadows=False)

for i in range(roi_start + 1, roi_end, 1):
    img = cv2.imread(f'{PATH}/input/in%06d.jpg' % i)
    IG = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    fgmask = fgbg.apply(IG, learningRate=0.01)


    ref_mask = cv2.imread(f'{PATH}/groundtruth/gt%06d.png' % i, cv2.IMREAD_GRAYSCALE)


    TP, TN, FP, FN = calculate_metrics(fgmask, ref_mask, TP, TN, FP, FN)

    # cv2.imshow('frame', fgmask)
    # cv2.waitKey(3)

P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)
print(f'{np.mean.__name__} precision: ', P, 'Recall: ', R, 'F1: ', F1)

mean precision:  0.23086038399533068 Recall:  0.974011048471966 F1:  0.37325237964264874
